# Load Data Into SQL Database

Load the datasets (stores in .csv files) into SQL Database called eda_project.

In [4]:
import sqlite3
import sqlalchemy 
from sqlalchemy import create_engine
from datetime import datetime, timedelta,date
import pandas as pd
import csv
import warnings

Take six months data from the MTA turstile dataset (August 2020 to March 2021) and load it in .CSV file

In [5]:

#function to generate data from augest 2020 to august 2021
def generate_weeks(starting_date):
    weeks=[]
    x=datetime.strptime(starting_date, '%d-%m-%Y')
    date_time_str = '2021-03-01'
    date_time_obj =datetime.strptime(date_time_str, '%Y-%m-%d')
    end_date = datetime(date_time_obj.year, date_time_obj.month, date_time_obj.day)
    while x<end_date:
        strr=x.strftime("%x")
        strr= strr[6:]+strr[0:2]+strr[3:5]
        weeks.append(strr)
        x = x + timedelta(days=7)

    return weeks
weeks_list= generate_weeks('01-08-2020')

In [6]:
#function to extract data from MTA turnstile dataset and save it .CSV file

import requests
def load_data(weeks):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in weeks:
        url_file = url.format(week_num)
        req = requests.get(url_file)
        url_content = req.content
        csv_file = open('turnstile.csv', "ab")
        csv_file.write(url_content)
    csv_file.close()
load_data(weeks_list)

Obtain the equipment(aka elevators,escalators,turstiles) data from MTA API

In [9]:
import csv
api_key='qmnIE64nrW7T0MQ09kf0A18VUw2EnigW561U3DLS'

def get_json_into_csv(url, c_file_name):
    header = { "x-api-key": api_key}
    a = requests.get(url, headers=header)
    a= a.json()
    data_file = open(c_file_name, "a",newline='')
    csv_writer = csv.writer(data_file)
 
    count = 0
    for data in a:
        if count == 0:
            header = data.keys()
            csv_writer.writerow(header)
            count += 1
        csv_writer.writerow(data.values())
 
    data_file.close()

In [11]:
get_json_into_csv("https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fnyct_ene_equipments.json",'Equipment.csv')

In [12]:
get_json_into_csv("https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fnyct_ene.json",'e_outages.csv')

In [13]:
get_json_into_csv("https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fnyct_ene_upcoming.json",'upcoming_outages.csv')

### Load .CSV files into SQL Database

In [20]:

data=[]
warnings.filterwarnings('ignore')
connection = sqlite3.connect('eda_project.db')
cursor = connection.cursor()
def load_into_db(t, insert_statement):
    #connection.open()
    with open(t, 'r', encoding='latin1') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)
        data = list()
        for row in reader:
            data.append(row)
    cursor.executemany(insert_statement, data)
    connection.commit()
    connection.close()

1- MTA Turnstile Data

In [3]:
insert_turnstile = r"INSERT INTO turnstile VALUES(?,?,?,?,?,?,?,?,?,?,?)"
load_into_db('turnstie.csv',insert_turnstile)

2- MTA Elevator Data

In [21]:
insert_elv = r"INSERT INTO elevator VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
load_into_db('Equipment.csv',insert_elv)

3- weather data

In [ ]:
insert_weather = r"INSERT INTO nyc_weather VALUES(?,?,?,?,?,?,?,?,?,?,?)"
load_into_db('turnstie.csv',insert_weather)

# EDA

Working with pandas

In [22]:
engine = create_engine("sqlite:///eda_project.db")


In [23]:
all_tables = engine.table_names()
all_tables

['elevator', 'nyc_weather', 'turnstile']

In [24]:
equipment= pd.read_sql('SELECT * FROM elevator;', engine)
equipment

,station,borough,trainno,equipmentno,equipmenttype,serving,ADA,isactive,nonNYCT,shortdescription,linesservedbyelevator,elevatorsgtfsstopid,elevatormrn,stationcomplexid,nextadanorth,nextadasouth,redundant,busconnections,alternativeroute
0,1 Av,MN,L,EL292,EL,E 14 St and Avenue A (NW corner) to 8 Av-bound...,Y,Y,N,street to 8 Av-bound platform,L,L06,119,119,602,120,0,"M15, M15 SBS, M14A SBS, M14D SBS",If you are on the street: Take a West Side-bou...
1,1 Av,MN,L,EL293,EL,E 14 St and Avenue A (SW corner) to Canarsie-b...,Y,Y,N,street to Brooklyn-bound platform,L,L06,119,119,602,120,0,"M15, M15 SBS, M14A SBS, M14D SBS",If you are on the street: Take a westbound M14...
2,125 St,MN,4/5/6,EL125,EL,125 St &amp; Lexington Ave (NE corner) to mezz...,Y,Y,N,street to mezzanine,4/5/6,621,392,392,"604, 4 / 434, 5 / 371, 6","610, 4, 5 / 612, 6",0,"M35, M60 SBS, M100, M101, M103, Bx15, M98 (wee...",If you are on the street: \nTake a downtown M1...
3,125 St,MN,4/5/6,EL126,EL,mezzanine to platforms for service in both dir...,Y,Y,N,mezzanine to platforms,4/5/6,621,392,392,"604, 4 / 434, 5 / 371, 6","610, 4, 5 / 612, 6",0,"M35, M60 SBS, M100, M101, M103, Bx15, M98 (wee...",If you are on the mezzanine: \nUse elevator to...
4,125 St,MN,A/B/C/D,EL142,EL,125 St &amp; St Nicholas Ave (SW corner) to me...,Y,Y,N,street to mezzanine,A/B/C/D,A15,153,153,"605, A, C / 604, B, D",614,0,"M3, M10, M60 SBS, M100, M101, M104, Bx15",If you are on the street:\nTake a Fort George-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,WTC Cortlandt St,MN,1/E/R/W,EL14X,EL,Greenwich St to Oculus Balcony with access to ...,Y,Y,Y,street to uptown 1,1,R25/E01/138,21/171/328,328,327,635,1,"M9, M20, M55, SIM1, SIM2, SIM4, SIM15, SIM32, ...",Use another elevator on this level.
624,WTC Cortlandt St,MN,1,EL712,EL,mezzanine at Oculus main level to uptown 1 pla...,Y,Y,N,mezzanine to uptown 1,1,138,328,328,327,635,1,"M9, M20, M55, SIM1, SIM2, SIM4, SIM15, SIM32, ...",Use another elevator on this level.
625,WTC Cortlandt St,MN,1,EL713,EL,mezzanine at Oculus main level to downtown 1 p...,Y,Y,N,mezzanine to downtown 1,1,138,328,328,327,635,1,"M9, M20, M55, SIM1, SIM2, SIM4, SIM15, SIM32, ...",Use another elevator on this level.
626,WTC Cortlandt St,MN,1,EL736,EL,Greenwich Street (near southwest corner of Gre...,Y,N,N,street to downtown 1,1,138,328,328,327,635,1,"M9, M20, M55, SIM1, SIM2, SIM4, SIM15, SIM32, ...",Use another elevator on this level.


### MTA Data Exploration

In [35]:
turnstile_data= pd.read_sql('SELECT * FROM turnstile;', engine)
turnstile_data

,C/A\t,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/25/2020,00:00:00,REGULAR,7436571,2527896
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/25/2020,04:00:00,REGULAR,7436572,2527900
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/25/2020,08:00:00,REGULAR,7436581,2527907
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/25/2020,12:00:00,REGULAR,7436611,2527942
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/25/2020,16:00:00,REGULAR,7436681,2527962
...,...,...,...,...,...,...,...,...,...,...,...
6509926,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/26/2021,04:00:00,REGULAR,5554,544
6509927,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/26/2021,08:00:00,REGULAR,5554,544
6509928,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/26/2021,12:00:00,REGULAR,5554,544
6509929,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,02/26/2021,16:00:00,REGULAR,5554,544


In [36]:
turnstile_data = turnstile_data.rename(columns={'C/A\t': 'C/A'})

In [37]:
turnstile_data['turnstile'] = turnstile_data['C/A'] + '-' + turnstile_data['UNIT'] + '-' + turnstile_data['SCP'] + '-' + turnstile_data['STATION'] 

In [40]:
# Checking columns names

turnstile_data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS', 'turnstile'],
      dtype='object')

In [49]:
# Eliminate spaces at columns names

turnstile_data.columns = turnstile_data.columns.str.strip()
turnstile_data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS', 'turnstile'],
      dtype='object')

In [63]:
#explore the data types
turnstile_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6509931 entries, 0 to 6509930
Data columns (total 12 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   C/A        object
 1   UNIT       object
 2   SCP        object
 3   STATION    object
 4   LINENAME   object
 5   DIVISION   object
 6   DATE       object
 7   TIME       object
 8   DESC       object
 9   ENTRIES    object
 10  EXITS      object
 11  turnstile  object
dtypes: object(12)
memory usage: 596.0+ MB


In [68]:
turnstile_data['DATE'] = pd.to_datetime(turnstile_data['DATE'],dayfirst=True, errors='coerce')

In [79]:
mel_count=turnstile_data['TIME'].str.contains(' ').sum()
if mel_count>0:
    print ("There are {m} Mels".format(m=mel_count))
else: 
    print('no space')

no space


In [74]:
turnstile_data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,turnstile
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-07-25,00:00:00,REGULAR,7436571,2527896,A002-R051-02-00-00-59 ST
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-07-25,04:00:00,REGULAR,7436572,2527900,A002-R051-02-00-00-59 ST
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-07-25,08:00:00,REGULAR,7436581,2527907,A002-R051-02-00-00-59 ST
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-07-25,12:00:00,REGULAR,7436611,2527942,A002-R051-02-00-00-59 ST
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2020-07-25,16:00:00,REGULAR,7436681,2527962,A002-R051-02-00-00-59 ST


In [75]:
# Creating DATETIME as datetime64 column that concatenate "DATE" and "TIME"

turnstile_data['DATETIME'] = pd.to_datetime(str(turnstile_data.DATE) + " " + turnstile_data.TIME,format="%Y-%m-%d %H:%M:%S")

ValueError: time data 0         2020-07-25
1         2020-07-25
2         2020-07-25
3         2020-07-25
4         2020-07-25
             ...    
6509926   2021-02-26
6509927   2021-02-26
6509928   2021-02-26
6509929   2021-02-26
6509930   2021-02-26
Name: DATE, Length: 6509931, dtype: datetime64[ns] 00:00:00 doesn't match format specified

In [47]:
#Create a column that called "NAMEOFDAY"
turnstile_data["NAMEOFDAY"]= turnstile_data["DATETIME"].dt.day_name()


AttributeError: Can only use .dt accessor with datetimelike values

### MTA Turnstile Data Cleaning

In [ ]:
#Checking null values based on columns

mta.isnull().sum()

make sure that EXITIS is less than ENTRIES because the emergency exits in the station doesn't have turnstile

In [39]:
print('Descriptions of entries:')
print(turnstile_data['ENTRIES'].describe())
print('')
print('Descriptions of exits:')
print(turnstile_data['EXITS'].describe())

Descriptions of entries:
count     6509931
unique    3618015
top             0
freq        91247
Name: ENTRIES, dtype: int64

Descriptions of exits:
count     6509931
unique    3372507
top             0
freq       256580
Name: EXITS, dtype: int64


Drop unnecessary columns - DESC, EXITS

In [26]:
del turnstile_data['EXITS']
del turnstile_data['DESC']

Obtain the number of entries at each station

In [ ]:
#Checking Unique Station count

len(mta["STATION"].unique())

In [ ]:
entries_per_station = turnstile_data.groupby('STATION')